# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# Build pipeline that draws the detected lane lines on the original images

def process_image(image):
    
    # Mask the image with color selection filter    
    color_filtered_image = color_selection(image)
    # Convert the RGB image to grayscale
    gray_image = grayscale(color_filtered_image)
    # Blur the image with Gaussian filter
    blurred_image = gaussian_blur(gray_image, kernel_size=15)
    # Extrct the edge using Canny edge filter
    edge_image = canny(blurred_image, low_threshold=50, high_threshold=150)
    # Return the coordinates of the vertices of ROI
    vertices = select_vertices(edge_image)
    # Mask the edge_image with ROI
    roi_image = region_of_interest(edge_image, vertices)
    # Detect the straight lines using the Hough transform
    lines = hough_lines(roi_image, rho=1, theta=np.pi/180, threshold=20, min_line_len = 20, max_line_gap = 300)
    # Merge the detected lines into two lines: left line and right line
    left_lane, right_lane = average_line(lines)
    # Detect the (x1,y1) , (x2, y2) points for each line and plot them on blank image
    lane_image = lane_points(image, left_lane, right_lane)
    # Overlay the lane line on the origin RGB image
    lane_image_scene = cv2.addWeighted(image, 1.0, lane_image, 0.50, 0.0)
    # return the overlay image
    return lane_image_scene

## Helper Functions

In [16]:
# Mask the pavement images with lane colors: yellow and white  
def color_selection(img): 
    # white color selection
    lower = np.uint8([200, 200, 200])
    upper = np.uint8([255, 255, 255])
    white_pixels = cv2.inRange(img, lower, upper)
    # yellow color selection
    lower = np.uint8([190, 190, 0])
    upper = np.uint8([255, 255, 255])
    yellow_pixels = cv2.inRange(img, lower, upper)
    # mask the image
    masked_pixels = cv2.bitwise_or(white_pixels, yellow_pixels)
    masked_image = cv2.bitwise_and(img, img, mask = masked_pixels)
    return masked_image

In [17]:
# Covert the RGB image into gray image
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [18]:
# Apply gaussian blur filter
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [19]:
# Apply edge detection
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

In [20]:
# Mask region of interest
def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Select vertices for the mask
def select_vertices(img):
    # first, define the polygon by vertices
    rows, cols = img.shape[:2]
    bottom_left  = [cols*0.1, rows*0.95]
    top_left     = [cols*0.4, rows*0.6]
    bottom_right = [cols*0.9, rows*0.95]
    top_right    = [cols*0.6, rows*0.6] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return vertices

In [21]:
# Apply Hough transform to detect the line
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

# Draw lines on the image
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    

In [22]:
# Merge the detected lines into two lines: left and right
def average_line(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (float(y2)-float(y1))/(float(x2)-float(x1))
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    # return two lanes in the format of slope and intercept
    return left_lane, right_lane 

# convert the y = mX +b to x1, y1, x2, y2
def make_line_points(y1, y2, line):

    if line is None:
        return None
    
    slope, intercept = line
    
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

# Plot the lane line points
def lane_points(img, left_lane, right_lane):
    lane_image = np.zeros_like(image)
    
    y1 = img.shape[0] # bottom of the image
    y2 = y1*0.6         # slightly lower than the middle
    left_lane_points  = make_line_points(y1, y2, left_lane)
    right_lane_points  = make_line_points(y1, y2, right_lane)
    
    cv2.line(lane_image, left_lane_points[0], left_lane_points[1], color=[0, 255, 0], thickness=10)
    cv2.line(lane_image, right_lane_points[0], right_lane_points[1], color=[0, 255, 0], thickness=10)

    return lane_image
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import os
import cv2
import math

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 15.0, (540, 960))
cap = cv2.VideoCapture('test_videos/solidYellowLeft.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    RGB_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    processed_image = process_image(RGB_img)
    processed_image_RGB = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    out.write(processed_image_RGB)
    cv2.imshow('frame', processed_image_RGB)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [25]:
def process_image(image):
    
    color_filtered_image = color_selection(image)
    gray_image = grayscale(color_filtered_image)
    blurred_image = gaussian_blur(gray_image, kernel_size=15)
    edge_image = canny(blurred_image, low_threshold=50, high_threshold=150)
    vertices = select_vertices(edge_image)
    roi_image = region_of_interest(edge_image, vertices)
    lines = hough_lines(roi_image, rho=1, theta=np.pi/180, threshold=20, min_line_len = 20, max_line_gap = 300)
    left_lane, right_lane = average_line(lines)
    lane_image = lane_points(image, left_lane, right_lane)
    lane_image_scene = cv2.addWeighted(image, 1.0, lane_image, 0.50, 0.0)

    return result

Let's try the one with the solid white lane on the right first ...

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.
